In [1]:
import sys
sys.path.append('/Users/Udit/Dev/fundas-app')
print(sys.path)
from Screener import Screener
import pandas as pd
from sqlalchemy import create_engine
import time
import DataImporter
from PathResolver import resolve_data
import datetime
import app.util.DataAccess
import concurrent.futures
import datetime
import click
from flask import jsonify
from app.app import create_app
from app.blueprints.api.models import CompanyInfo, LatestStandalone, LatestConsolidated, Technicals, TechnicalsHistorical
from app.extensions import db
from app.util import ScreenerUtil, DataImporter, DataAccess
from app.util.PathResolver import resolve_data
import app.util.DBUtil as DBUtil



app = create_app()
db.app = app


['', '/Users/Udit/Dev/fundas-app/.venv/lib/python36.zip', '/Users/Udit/Dev/fundas-app/.venv/lib/python3.6', '/Users/Udit/Dev/fundas-app/.venv/lib/python3.6/lib-dynload', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/Users/Udit/Dev/fundas-app/.venv/lib/python3.6/site-packages', '/Users/Udit/Dev/fundas-app/.venv/lib/python3.6/site-packages/IPython/extensions', '/Users/Udit/.ipython', '/Users/Udit/Dev/fundas-app']


/Users/Udit/Dev/fundas-app/app/extensions.py:7: FlaskWTFDeprecationWarning: "flask_wtf.CsrfProtect" has been renamed to "CSRFProtect" and will be removed in 1.0.
  csrf = CsrfProtect()
/Users/Udit/Dev/fundas-app/app/app.py:25: ExtDeprecationWarning: Importing flask.ext.cache is deprecated, use flask_cache instead.
  from flask.ext.cache import Cache


In [ ]:
t = Technicals.query.get('AFEL')

In [ ]:

query = db.session.query(TechnicalsHistorical).join(Technicals).filter(Technicals.symbol=='AFEL')

pd.read_sql(query.statement,db.session.bind)

In [13]:
def get_technicals_as_df(symbol):
    tech = get_technicals(symbol)
    query = db.session.query(TechnicalsHistorical).join(Technicals).filter(Technicals.symbol==tech.symbol)
    return pd.read_sql(query.statement, db.session.bind)


def get_technicals(symbol):
    tech = Technicals.query.get(symbol)
    current_datetime = datetime.datetime.utcnow()
    if not tech:
        print(f"Technical data not available for {symbol}")
        tech = update_technicals(symbol)
    else:
        d1_ts = time.mktime(current_datetime.timetuple())
        d2_ts = time.mktime(tech.updated_on.timetuple())
        time_elapsed = (d1_ts - d2_ts)/60
        print(f"{symbol} was updated {time_elapsed} minutes ago.")
        if time_elapsed > 5 * 24 * 7:
            tech = update_technicals(symbol)
    return tech

def update_technicals(symbol):
    ret_dict = DataAccess.get_technicals(symbol)
    DBUtil.bulk_insert_technical_data([ret_dict])
    return Technicals.query.get(symbol)
    

In [17]:
get_technicals_as_df('PNB')

Technical data not available for PNB
Key is : NSE/PNB
Persisting $1 entries
Created 1 <class 'app.blueprints.api.models.Technicals'>
Created 260 <class 'app.blueprints.api.models.TechnicalsHistorical'>


,created_on,updated_on,id,symbol,date,open,high,low,close,volume,hhv52,hhv_all_time
0,2017-12-25 05:34:16,2017-12-25 05:34:16,3203,PNB,2013-01-06,892.90,910.75,891.60,908.10,689119.0,910.75,910.75
1,2017-12-25 05:34:16,2017-12-25 05:34:16,3204,PNB,2013-01-13,889.10,893.45,877.25,880.75,349290.0,910.75,910.75
2,2017-12-25 05:34:16,2017-12-25 05:34:16,3205,PNB,2013-01-20,891.20,905.00,889.05,894.30,440484.0,910.75,910.75
3,2017-12-25 05:34:16,2017-12-25 05:34:16,3206,PNB,2013-01-27,852.20,877.00,845.20,873.35,393727.0,910.75,910.75
4,2017-12-25 05:34:16,2017-12-25 05:34:16,3207,PNB,2013-02-03,912.00,916.00,893.35,899.85,2050627.0,916.00,916.00
5,2017-12-25 05:34:16,2017-12-25 05:34:16,3208,PNB,2013-02-10,880.00,881.85,858.90,862.55,590126.0,916.00,916.00
6,2017-12-25 05:34:16,2017-12-25 05:34:16,3209,PNB,2013-02-17,846.00,854.70,838.05,845.85,497445.0,916.00,916.00
7,2017-12-25 05:34:16,2017-12-25 05:34:16,3210,PNB,2013-02-24,838.20,856.90,834.20,853.05,396944.0,916.00,916.00
8,2017-12-25 05:34:16,2017-12-25 05:34:16,3211,PNB,2013-03-03,791.00,804.90,781.15,788.65,710245.0,916.00,916.00
9,2017-12-25 05:34:16,2017-12-25 05:34:16,3212,PNB,2013-03-10,813.10,818.30,805.30,814.65,502611.0,916.00,916.00


In [18]:
db.session.close()